In [8]:
import numpy as np
import pandas as pd
import sympy as sp 
from scipy.optimize import minimize 
from scipy.optimize import differential_evolution

In [2]:
#Read in the dataset 'nonlinsep.txt', storing the datapoints in the form of list 
f= open('nonlinsep.txt','r')
data = []
for line in f.readlines():
    datapoint = line.strip().split(',')
    datapoint = list(map(float,datapoint))
    data.append(datapoint)

In [3]:
#Check our dataset:
#data

In [4]:
#polynomial transformation
data_transform = []
for i in range(len(data)):
    new_space = [1,data[i][0],data[i][1],data[i][0]**2,data[i][0]*data[i][1],data[i][1]**2,data[i][1]*data[i][0]]
    data_transform.append(new_space)

In [7]:
#data_transform[0:10]

In [ ]:
#polynomial kernel 
def polynomial_kernel(xi,xj):
    k = 1 + np.dot(xi,xj) + (np.dot(xi,xj))**2
    return k 

In [17]:
lambdas = []
for i in range(len(data)):
    globals()['lambda'+str(i+1)] = sp.symbols('lambda{}'.format(i+1))
    lambdas.append(globals()['lambda'+str(i+1)])

In [ ]:
sigma_ll_yy_xx = 0 
for i in range(len(data)):
    for j in range(len(lambdas)):
        sigma_ll_yy_xx += lambdas[i] * lambdas[j] * data[i][2] * data[j][2] * polynomial_kernel(data[i][0:2],data[j][0:2])


In [ ]:
def lagrange_min(x):
    sigma_lambda = 0
    for lambda_i in lambdas:
        sigma_lambda += lambda_i

    lagrange = (sigma_lambda)-(1/2)*sigma_ll_yy_xx 
    
    zipped = zip(lambdas,x)
    
    return lagrange.subs(zipped)

In [ ]:
#KKT:
def sigma_lambda_y(x):
    sigma_lambda_y = 0
    for i in range(len(lambdas)):
        sigma_lambda_y  += lambdas[i]*data[i][2]-0  
    zipped = zip(lambdas,x)
    return sigma_lambda_y.subs(zipped)

x0=[2]*100
lamb_positive = []
for i in range(len(lambdas)):
    globals()['bound'+str(i+1)]= (0,np.inf)
    lamb_positive.append(globals()['bound'+str(i+1)])

lamb_positive = tuple(lamb_positive)

In [ ]:
#Minimize the Lagrangian function:
solution_lambda = minimize(lagrange_min,x0,tol=1e-8,bounds=lamb_positive,constraints=[{'type':'eq','fun':sigma_lambda_y}])

In [ ]:
#Second attempt
solution_lambda2 = minimize(lagrange_min,x0,tol=1e-8,method='nelder-mead')

In [ ]:
#Third attempt, using differential_evolution
solution_lambda3 = differential_evolution(lagrange_min,bounds=lamb_positive)

In [ ]:
#W*

w_star = []

for i in range(len(data)):
    lambdai_yi_xi = [(list(solution_lambda.values())[i]*data[i][2])*x for x in data_transform[i][0:6]]
    w_star = np.sum([w_star,lambdai_yi_xi],axis=0)

w_star = w_star.tolist()

In [ ]:
#b* 